In [29]:
# Start Database or reconnect to it
import sqlite3

try:
    conn = sqlite3.connect('shark_tables.db')
    cursor = conn.cursor()
    print("Successful Connection to database")
except:
    print("Failed to Connect")

Successful Connection to database


In [30]:
# Grab the csv files
import pandas as pd
try:
    activity_df = pd.read_csv("GOdak/activity.csv", low_memory=False)
    date_df = pd.read_csv("GOdak/date.csv", low_memory=False)
    location_df = pd.read_csv("GOdak/location.csv", low_memory=False)
    species_df = pd.read_csv("GOdak/species.csv", low_memory=False)
    type_df = pd.read_csv("GOdak/type.csv", low_memory=False)
    victimology_df = pd.read_csv("GOdak/victimology.csv", low_memory=False)
    fatality_df = pd.read_csv("GOdak/fatality.csv", low_memory=False)
    print("Successfully Read Data File")
    
except:
    print("Failed to Read Data File")

Successfully Read Data File


In [31]:
# Convert dataframes into database
activity_df.to_sql('Activity',conn,if_exists='replace',index=False)
date_df.to_sql('Date',conn,if_exists='replace',index=False)
location_df.to_sql('Location',conn,if_exists='replace',index=False)
species_df.to_sql('Species',conn,if_exists='replace',index=False)
type_df.to_sql('Type',conn,if_exists='replace',index=False)
victimology_df.to_sql('Victimology',conn,if_exists='replace',index=False)
fatality_df.to_sql('Fatality',conn,if_exists='replace',index=False)
conn.commit()
r_df = pd.read_sql("select * from Activity",conn)
print(r_df)
#conn.close()

       Case Number                    Activity
0       2020.02.05              Board_Activity
1     2020.01.30.R           Swimming_Activity
2       2020.01.17              Board_Activity
3       2020.01.16              Board_Activity
4       2020.01.13              Board_Activity
...            ...                         ...
5930       ND.0005        Free_Diving_Activity
5931       ND.0004  Shellfish_Hunting_Activity
5932       ND.0003           Swimming_Activity
5933       ND.0002            Unknown_Activity
5934       ND.0001           Swimming_Activity

[5935 rows x 2 columns]


In [34]:
# Get all tables from database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Activity',), ('Date',), ('Location',), ('Species',), ('Type',), ('Victimology',), ('Fatality',)]


In [26]:
# Combine tables
conn.execute("""
CREATE TABLE 
CompleteData
AS
SELECT
Activity.[Case Number],
Date.Date,
Date.Year,
Date.Month,
Date.Day,
Location.Country,
Activity.Activity,
Species.Species,
Type.Type,
Victimology.[People Involved],
Victimology.Sex,
Victimology.[Original Name]
FROM
Activity
LEFT JOIN Location ON Location.[Case Number] = Activity.[Case Number]
LEFT JOIN Date ON Date.[Case Number] = Activity.[Case Number]
LEFT JOIN Species ON Species.[Case Number] = Activity.[Case Number]
LEFT JOIN Type ON Location.[Case Number] = Activity.[Case Number]
LEFT JOIN Victimology ON Victimology.[Case Number] = Activity.[Case Number]
LEFT JOIN Fatality ON Fatality.[Case Number] = Activity.[Case Number]
""")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
#r_df = pd.read_sql("select * from CompleteData",conn)
#print(r_df)

[('Activity',), ('Date',), ('Location',), ('Species',), ('Type',), ('Victimology',), ('CompleteData',)]


In [27]:
cursor.execute("SELECT COUNT([Case Number]) FROM CompleteData LIMIT 5")
print(cursor.fetchall())


[(37420175,)]


In [33]:
# Drop Test data
conn.execute("""
DROP TABLE CompleteData
""")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Activity',), ('Date',), ('Location',), ('Species',), ('Type',), ('Victimology',), ('Fatality',)]


In [28]:
# Close Database
conn.commit()
conn.close()